# Determining which parts of the city attract which types of food truck

### Exploratory Data Analysis

1. Import packages and read dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# reading the CSV file
df = pd.read_csv('Mobile_Food_Facility_Permit.csv')

In [2]:
df.head()

,locationid,Applicant,FacilityType,cnn,LocationDescription,Address,blocklot,block,lot,permit,...,Approved,Received,PriorPermit,ExpirationDate,Location,Fire Prevention Districts,Police Districts,Supervisor Districts,Zip Codes,Neighborhoods (old)
0,735318,Ziaurehman Amini,Push Cart,30727000,MARKET ST: DRUMM ST intersection,5 THE EMBARCADERO,0234017,0234,017,15MFF-0159,...,NaN,20151231,0,03/15/2016 12:00:00 AM,"(37.794331003246846, -122.39581105302317)",4.0,1.0,10.0,28855.0,6.0
1,1612654,Sunset Mercantile/Outer Sunset Farmers Market ...,Truck,1835000,37TH AVE: QUINTARA ST to RIVERA ST (2100 - 2199),2155 37TH AVE,2094006,2094,006,22MFF-00031,...,NaN,20220421,0,11/15/2022 12:00:00 AM,"(37.74732654654123, -122.49628067270531)",1.0,8.0,3.0,29491.0,35.0
2,1658690,Bonito Poke,Truck,3528000,CALIFORNIA ST: SANSOME ST to LEIDESDORFF ST (4...,430 CALIFORNIA ST,0239029,0239,029,22MFF-00070,...,11/09/2022 12:00:00 AM,20221109,0,11/15/2023 12:00:00 AM,"(37.793262206923096, -122.4017890913628)",4.0,1.0,10.0,28854.0,6.0
3,1652620,"Off the Grid Services, LLC",Truck,2953000,BERRY ST: 03RD ST to 04TH ST (100 - 199),185 BERRY ST,3803005,3803,005,22MFF-00036,...,11/22/2022 12:00:00 AM,20221019,0,11/15/2023 12:00:00 AM,"(37.77632714778992, -122.39179682107691)",6.0,2.0,9.0,28856.0,20.0
4,1652621,"Off the Grid Services, LLC",Truck,3143000,BROADWAY: DAVIS ST to FRONT ST (50 - 99),90 BROADWAY,0140007,0140,007,22MFF-00036,...,11/22/2022 12:00:00 AM,20221019,0,11/15/2023 12:00:00 AM,"(37.799260113502285, -122.39961794865545)",3.0,1.0,10.0,28860.0,6.0


2. Data understanding and cleaning

In [3]:
df.shape

(481, 29)

In [7]:
for column in df.columns:
    print(column, df[column].isna().sum())

locationid 0
Applicant 0
FacilityType 8
cnn 0
LocationDescription 15
Address 0
blocklot 10
block 10
lot 10
permit 0
Status 0
FoodItems 6
X 33
Y 33
Latitude 0
Longitude 0
Schedule 0
dayshours 394
NOISent 481
Approved 130
Received 0
PriorPermit 0
ExpirationDate 23
Location 0
Fire Prevention Districts 34
Police Districts 34
Supervisor Districts 34
Zip Codes 33
Neighborhoods (old) 34


In [20]:
df.groupby('FoodItems').size().reset_index(name='count').sort_values('count', ascending=False)

,FoodItems,count
33,Cold Truck: Sandwiches: Noodles: Pre-packaged...,38
15,Burgers: melts: hot dogs: burritos:sandwiches:...,37
32,Cold Truck: Pre-packaged sandwiches: snacks: f...,33
30,Cold Truck: Hamburger: cheeseburgers: hot dogs...,23
65,Ice Cream: Pre-Packaged Chips: Candies: Bottle...,21
...,...,...
63,Ice Cream & Waffle Cones,1
62,Hotdogs: chips: soda: nuts: crackers,1
61,Hot dogs: sausages: cheesesteaks: chips: drinks,1
60,Hot dogs: condiments: soft pretzels: soft drin...,1


In [18]:
foodtype=[]
for row in df.groupby('FoodItems').size().reset_index(name='count').sort_values('count', ascending=False)['FoodItems']:
    foodtype.append(row)

In [19]:
foodtype

['Cold Truck: Sandwiches: Noodles:  Pre-packaged Snacks: Candy: Desserts Various Beverages',
 'Burgers: melts: hot dogs: burritos:sandwiches: fries: onion rings: drinks',
 'Cold Truck: Pre-packaged sandwiches: snacks: fruit: various beverages',
 'Cold Truck: Hamburger: cheeseburgers: hot dogs: hot sandwiches: cold sandwiches: egg muffins: cup of noodles: corn dogs: canned soup: coffee: hot cocoa: hot tea: gatorade: juice: milk: soda: water: fruits: fruit salad: rice pudding: yogurt: candy bars: chips: cookies: donuts: granola bars: muffins',
 'Ice Cream: Pre-Packaged Chips: Candies: Bottled Water & Canned SODA',
 'COLD TRUCK. Deli: bbq chicken skewer: Chinese spring roll: Chinese fried rice/noodle: fried chicken leg/wing: bbq chicken sandwich: chicken cheese burger: burrito: lumpia. Snack: sunflower seeds: muffins: chips: snickers: kit-kat: 10 types of chocolate. Drinks: Coke: 7-Up: Dr. Pepper: Pepsi: Redbull: Vitamin Water: Rockstar: Coconut Juice: Water. Hot drinks: coffee: tea.',
 '

3. Manually reassigning food types to new labels

In [75]:
# food types are extremely varied: going to assign categories for each to fit into
# e.g. Mexican, Japanese, Breakfast, Hot Dogs, ...
foodtier2 = dict.fromkeys(foodtype, None)
for item in foodtype:
    if 'COLD TRUCK' in item.upper() or 'CRACKERJACKS' in item.upper() or 'KETTLE' in item.upper():
        foodtier2[item] = 'COLD TRUCK'
    elif 'EVERYTHING' in item.upper() or 'VARIES' in item.upper() or 'ALL TYPES' in item.upper():
        foodtier2[item] = 'OTHER'
    elif 'FILIPINO' in item.upper():
        foodtier2[item] = 'FILIPINO'
    elif ('MEXICAN' in item.upper() or 'TACO' in item.upper() or 'BURRITO' in item.upper() or 'COCHINITA' in item.upper() or 'FRUIT SALADS: FRUIT DRINKS' in item.upper() or 'MANGONEADA' in item.upper()) and 'MASALA' not in item.upper() and 'JERK' not in item.upper():
        foodtier2[item] = 'MEXICAN'
    elif ('INDIAN' in item.upper() or 'MASALA' in item.upper() or 'PULAO' in item.upper() or 'CHAI' in item.upper()) and 'BBQ PORK' not in item.upper():
        foodtier2[item] = 'INDIAN & SOUTH ASIAN'
    elif 'BRAZILIAN' in item.upper():
        foodtier2[item] = 'BRAZILIAN'
    elif 'PERUVIAN' in item.upper() or 'LOMO SALTADO' in item.upper():
        foodtier2[item] = 'PERUVIAN'
    elif 'JERK' in item.upper() or 'PLANTAIN' in item.upper():
        foodtier2[item] = 'JAMAICAN'
    elif 'CUBAN' in item.upper():
        foodtier2[item] = 'CUBAN'
    elif 'JAPANESE' in item.upper():
        foodtier2[item] = 'JAPANESE'
    elif 'VEGAN' in item.upper():
        foodtier2[item] = 'VEGAN'
    elif 'GYRO' in item.upper():
        foodtier2[item] = 'GYRO'
    elif 'POKE BOWL' in item.upper():
        foodtier2[item] = 'POKE BOWL'
    elif 'HOT DOG' in item.upper() or 'HOTDOG' in item.upper() or 'BURGER' in item.upper():
        foodtier2[item] = 'HOT DOGS & BURGERS'
    elif 'ICE CREAM' in item.upper():
        foodtier2[item] = 'ICE CREAM'
    elif 'PASTRY' in item.upper() or 'PASTRIES' in item.upper() or 'BREAKFAST' in item.upper() or 'CROISSA' in item.upper():
        foodtier2[item] = 'BREAKFAST'
    elif 'PIZZA' in item.upper():
        foodtier2[item] = 'PIZZA'
    elif 'PERUVIAN' in item.upper():
        foodtier2[item] = 'PERUVIAN'
    elif 'BAO' in item.upper():
        foodtier2[item] = 'BAO'
    elif 'ACAI' in item.upper() or 'FRUIT DRINK' in item.upper() or 'FRUITS' in item.upper() or 'SMOOTHIE' in item.upper():
        foodtier2[item] = 'SMOOTHIE'
    elif ('NOODLE' in item.upper() or 'RICE BOWL' in item.upper()) and 'BBQ PORK' not in item.upper():
        foodtier2[item] = 'NOODLES & RICE BOWLS'
    elif 'RIB' in item.upper() or 'FRIED CHICKEN' in item.upper() or 'MEATLOAF' in item.upper() or 'BBQ' in item.upper():
        foodtier2[item] = 'AMERICAN'
    elif 'CHEESESTEAK' in item.upper():
        foodtier2[item] = 'CHEESESTEAK'
    else:
        foodtier2[item] = 'OTHER'
    

In [76]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(pd.DataFrame.from_dict(foodtier2, orient = 'index', columns = ['FoodTier2']))

                                                               FoodTier2
Cold Truck: Sandwiches: Noodles:  Pre-packaged ...            COLD TRUCK
Burgers: melts: hot dogs: burritos:sandwiches: ...               MEXICAN
Cold Truck: Pre-packaged sandwiches: snacks: fr...            COLD TRUCK
Cold Truck: Hamburger: cheeseburgers: hot dogs:...            COLD TRUCK
Ice Cream: Pre-Packaged Chips: Candies: Bottled...             ICE CREAM
COLD TRUCK. Deli: bbq chicken skewer: Chinese s...            COLD TRUCK
Cold Truck: Soft drinks: cup cakes: potato chip...            COLD TRUCK
Cold Truck: Corn Dogs: Noodle Soups: Candy: Pre...            COLD TRUCK
Hot dogs: condiments: soft pretzels: soft drink...    HOT DOGS & BURGERS
Cold Truck: Breakfast: Sandwiches: Salads: Pre-...            COLD TRUCK
Cold Truck: Hot/Cold Sandwiches: Water: Soda: J...            COLD TRUCK
Bonito Poke Bowls & Various Drinks                             POKE BOWL
Cold Truck: Burrito: Corn Dog: Salads: Sandwich... 

In [78]:
# now want to create an even more general category
# e.g. Mexican, Peruvian, Cuban would all become Latin
foodtier1_list = []
for item in foodtier2.values():
    foodtier1_list.append(item)

In [79]:
foodtier1 = dict.fromkeys(foodtier1_list, None)

In [85]:
for item in foodtier1_list:
    if 'MEXICAN' in item.upper() or 'PERUVIAN' in item.upper() or 'CUBAN' in item.upper() or 'BRAZILIAN' in item.upper():
        foodtier1[item] = 'LATIN'
    elif 'BAO' in item.upper() or 'JAPANESE' in item.upper() or 'NOODLE' in item.upper():
        foodtier1[item] = 'EAST ASIAN'
    elif 'INDIAN' in item.upper() or 'FILIPINO' in item.upper():
        foodtier1[item] = 'SOUTH ASIAN'
    elif 'JAMAICAN' in item.upper():
        foodtier1[item] = 'CARIBBEAN'
    elif 'AMERICAN' in item.upper() or 'HOT DOG' in item.upper() or 'ICE CREAM' in item.upper() or 'BREAKFAST' in item.upper() or 'CHEESESTEAK' in item.upper() or 'VEGAN' in item.upper() or 'PIZZA' in item.upper() or 'POKE' in item.upper() or 'BREAKFAST' in item.upper():
        foodtier1[item] = 'AMERICAN'
    elif 'GYRO' in item.upper():
        foodtier1[item] = 'GREEK & MIDDLE EASTERN'
    else:
        foodtier1[item] = 'OTHER'
    

In [86]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(pd.DataFrame.from_dict(foodtier1, orient = 'index', columns = ['FoodTier1']))

                                   FoodTier1
COLD TRUCK                             OTHER
MEXICAN                                LATIN
ICE CREAM                           AMERICAN
HOT DOGS & BURGERS                  AMERICAN
POKE BOWL                           AMERICAN
OTHER                                  OTHER
BREAKFAST                           AMERICAN
PIZZA                               AMERICAN
JAMAICAN                           CARIBBEAN
PERUVIAN                               LATIN
BAO                               EAST ASIAN
NOODLES & RICE BOWLS              EAST ASIAN
SMOOTHIE                               OTHER
AMERICAN                            AMERICAN
INDIAN & SOUTH ASIAN             SOUTH ASIAN
FILIPINO                         SOUTH ASIAN
BRAZILIAN                              LATIN
GYRO                  GREEK & MIDDLE EASTERN
VEGAN                               AMERICAN
CUBAN                                  LATIN
CHEESESTEAK                         AMERICAN
JAPANESE  

4. Creating new columns in original dataframe for our Tier1 and Tier2 foodtype labels

In [87]:
df['FoodTier2']= df['FoodItems'].map(foodtier2)
df['FoodTier1']= df['FoodTier2'].map(foodtier1)

In [90]:
df.tail(10)

,locationid,Applicant,FacilityType,cnn,LocationDescription,Address,blocklot,block,lot,permit,...,PriorPermit,ExpirationDate,Location,Fire Prevention Districts,Police Districts,Supervisor Districts,Zip Codes,Neighborhoods (old),FoodTier2,FoodTier1
471,1587560,Brazuca Grill,Truck,5070000,EDDY ST: BUCHANAN ST to WEBSTER ST (1300 - 1399),1346 EDDY ST,0733014,0733,014,21MFF-00154,...,1,11/15/2022 12:00:00 AM,"(37.78186711757412, -122.42988524760261)",7.0,9.0,11.0,29490.0,41.0,COLD TRUCK,OTHER
472,1591827,Cochinita,Truck,3285000,BRYANT ST: 04TH ST \ I-80 E OFF RAMP to 05TH S...,617 BRYANT ST,3777055,3777,055,22MFF-00021,...,1,11/15/2022 12:00:00 AM,"(37.779051111573146, -122.39830388133372)",6.0,2.0,9.0,28856.0,34.0,MEXICAN,LATIN
473,1575192,Park's Catering,Truck,8483000,LOOMIS ST: BARNEVELD AVE \ MCKINNON AVE to OAK...,10 LOOMIS ST,5560100,5560,100,21MFF-00115,...,1,11/15/2022 12:00:00 AM,"(37.74378392338268, -122.40378882961045)",10.0,3.0,8.0,58.0,1.0,COLD TRUCK,OTHER
474,1587541,Brazuca Grill,Truck,1232000,23RD ST: START: 100-699 BLOCK to ILLINOIS ST (...,601 23RD ST,4232010,4232,010,21MFF-00154,...,1,11/15/2022 12:00:00 AM,"(37.755030726766726, -122.38453073422282)",10.0,3.0,8.0,28856.0,29.0,COLD TRUCK,OTHER
475,1575193,Park's Catering,Truck,8700000,MARIN ST: KANSAS ST to HWY 101 N ON RAMP (2500...,2525 MARIN ST,4343001D,4343,001D,21MFF-00115,...,1,11/15/2022 12:00:00 AM,"(37.74761607223309, -122.40197697572432)",10.0,3.0,8.0,58.0,1.0,COLD TRUCK,OTHER
476,1591829,Cochinita,Truck,4806101,DIVISADERO ST: MCALLISTER ST to GOLDEN GATE AV...,999 DIVISADERO ST,1156034,1156,034,22MFF-00021,...,1,11/15/2022 12:00:00 AM,"(37.778384300969044, -122.43878238856672)",15.0,5.0,11.0,29490.0,41.0,MEXICAN,LATIN
477,1591822,Cochinita,Truck,1329000,24TH ST: UTAH ST to POTRERO AVE (2600 - 2699),2601 24TH ST,4264001,4264,001,22MFF-00020,...,0,11/15/2022 12:00:00 AM,"(37.752822368121485, -122.4055319192157)",10.0,4.0,8.0,28859.0,19.0,MEXICAN,LATIN
478,1569152,Datam SF LLC dba Anzu To You,Truck,12463000,TAYLOR ST: BAY ST to NORTH POINT ST (2500 - 2599),2535 TAYLOR ST,0029007,0029,007,21MFF-00106,...,0,11/15/2022 12:00:00 AM,"(37.805885350100986, -122.41594524663745)",5.0,1.0,10.0,308.0,23.0,JAPANESE,EAST ASIAN
479,1587553,Brazuca Grill,Truck,197101,03RD ST: MARIN ST to ARTHUR AVE \ CARGO WAY (3...,3255 03RD ST,4377001,4377,001,21MFF-00154,...,1,11/15/2022 12:00:00 AM,"(37.748445338837655, -122.38687903197963)",10.0,3.0,8.0,58.0,1.0,COLD TRUCK,OTHER
480,1585474,"San Francisco Street Foods, Inc.",Push Cart,7038000,HOWARD ST: 03RD ST to 04TH ST (700 - 799),701 HOWARD ST,3734091,3734,091,21MFF-00145,...,1,11/15/2022 12:00:00 AM,"(37.783513351245354, -122.40081339110537)",6.0,2.0,9.0,28853.0,34.0,HOT DOGS & BURGERS,AMERICAN


### Mapping Visualization
Using Folium to plot city location against type of cuisine

5. Import packages and initialize map in SF

In [123]:
import folium

intro_map = folium.Map(location=[37.761229, -122.447213],zoom_start=13)

In [97]:
intro_map

6. Create icons to better visualize specific cuisines

In [131]:
# create dictionary and choose which icons and colours should represent each cuisine
food_icons = dict.fromkeys(df['FoodTier1'], None)

In [143]:
for key in food_icons.keys():
    if key == 'OTHER':
        food_icons[key] = ['bottle-water','lightgray']
    if key == 'AMERICAN':
        food_icons[key] = ['hotdog','beige']
    if key == 'LATIN':
        food_icons[key] = ['pepper-hot','red']
    if key == 'CARIBBEAN':
        food_icons[key] = ['lemon','lightgreen']
    if key == 'EAST ASIAN':
        food_icons[key] = ['bowl-rice','black']
    if key == 'GREEK & MIDDLE EASTERN':
        food_icons[key] = ['fish','lightblue']
    if key == 'SOUTH ASIAN':
        food_icons[key] = ['bowl-food','orange']

In [144]:
for index in df.index:
    for k,v in food_icons.items():
        if df['FoodTier1'][index]==k:
            folium.Marker(
                location=[eval(df['Location'][index])[0],eval(df['Location'][index])[1]],
                popup=df['FoodItems'][index],
                icon=folium.Icon(color=v[1],prefix='fa',icon=v[0]),
            ).add_to(intro_map)

In [145]:
intro_map

In [146]:
intro_map.save("foodtruck.html")